In [2]:
%load_ext autoreload
%autoreload 2

# organize the imports
import opentrons.execute
from opentrons import types 
import time
import numpy as np
import opentrons
from  opentrons.types import Location, Point

from opentrons_helper_class import * 

In [3]:
from imjoy_rpc import api
class ImJoyPlugin():
    async def setup(self):
        pass

    async def run(self, ctx):
        # for regular plugin
        # p = await api.getPlugin(my_plugin_source)

        # or for window plugin
        # await api.createWindow(src=my_plugin_source)
        await api.showDialog(src=my_plugin_source)

api.export(ImJoyPlugin())

<IPython.core.display.Javascript object>

<Future finished result=[]>

In [4]:
# Connect to the opentrons robot
protocol = opentrons.execute.get_protocol_api('2.10')
# setup the pipets
pipette = protocol.load_instrument('p300_single_gen2', 'right')
pipette_8 = protocol.load_instrument('p300_multi', 'left') # causes errors -> don't use two pipettes!!


In [5]:
'''
Connect to the microscope 
'''

from microscope_client import MicroscopeClient 
microscope = MicroscopeClient(host="21.3.2.3", port=5000)
#!pip3 install zeroconf
#!pip3 install requests


In [6]:
try:
    pipette.drop_off()
except:
    print("No pipette attached")
pipette.home()

No pipette attached


<InstrumentContext: p300_single_v2.1 in RIGHT>

In [7]:
# setup the labware in the trays
#plate_microscope = protocol.load_labware('corning_96_wellplate_360ul_flat', location='5')
#plate_reagents = protocol.load_labware('corning_24_wellplate_3.4ml_flat', location='9')
plate_reagents = protocol.load_labware('opentrons_24_tuberack_eppendorf_1.5ml_safelock_snapcap', location='10')


# Move light to the sample 
tiprack = protocol.load_labware('opentrons_96_tiprack_300ul',location='11')

# sample plate on microscope
# offset of the well plate in mm
plate_sample = protocol.load_labware('corning_96_wellplate_360ul_flat', location='3')

In [12]:
from opentrons.types import Location, Point

# set speed
#protocol.max_speeds['X'] = 200 # default = 400
#protocol.max_speeds['Y'] = 200 # default = 300

# ######
# 0 4 8
# 1 5 9
# 2 6 10 
# 3 7 11

# wells with reagents
i_ab_primary = 0
i_ab_secondary = 1
i_pbs = 2
i_bsa = 3
i_triton = 4
i_pfa = 5


# define timig
t_pfa = 15
t_triton = 10
t_bsa = 15
t_primary = 60
t_secondary = 60
t_fluo = 30*0

# move it down since opentrons expects the reagents to be furthter away from the ground
offset_z_reagents = -65

# define locations 
minimum_z_height = 155 # minimum height for arcs driven by the robot

# compute the positions
position_trash =  (365,350, 100)

position_idle =  (100,100, 150)#

In [9]:
#
protocol.home()
move_to_coord(pipette, position_trash, offset=(0,0,0), minimum_z_height=minimum_z_height)

Please add a pipette first!
Moving to: (365, 350, 100)


In [10]:
move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)

Please add a pipette first!
Moving to: (100, 100, 150)


In [13]:
from opentrons_client import OpentronsClient 
ot2 = OpentronsClient(host="21.3.2.4", port=31950)
ot2.toggle_light(False)
#ipot2.pipette_home(location="right")
#pos = ot2.positions()

{'on': False}

# Calibrate

In [ ]:
'''
calibrate tiprack
'''
#pitty, but moving the xy coordinates doesn't work
i_pipette=0
print(i_pipette)
offset_pipette_rack=(1,3,0)
tiprack.set_calibration(Point(*offset_pipette_rack))

i_pipette=pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack=offset_pipette_rack)
print(type(i_pipette))

In [35]:
'''
calibrate illumination poisiton
'''
#protocol.home()
#position_idle =  (100,300, 150)#
#move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)
#move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)
#microscope.home()
position_sample_light = (246, 99, 200) # height is necessary to reduce intensity
#Center of the center well
offset_x = 51990
offset_y = 47720
offset_z = microscope.position["z"]
move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
microscope.set_laser_led(0,1)
microscope.move((offset_x,offset_y,offset_z),absolute=True)
autofocus_dz = 1500
autofocus_Nz = 11
#offset_z = microscope.autofocus_coarse(dz=autofocus_dz, nz=autofocus_Nz)
time.sleep(1)
asdfasdf
microscope.set_laser_led(0,0)


microscope.set_laser_led(100,0)
offset_z = microscope.autofocus_coarse(dz=autofocus_dz, nz=autofocus_Nz)
microscope.set_laser_led(0,0)

Please add a pipette first!
Moving to: (246, 99, 200)
Setting Laser/LED to: 0 1


NameError: name 'asdfasdf' is not defined

In [ ]:
'''
Calibrate Wellposition for sample coordinates (96well plate)
pipette should dip in the wells on the microscope

'''
#pipette.home()
offset_z = microscope.position["z"]
home_position_microscope=(0,0,offset_z) # less vibration in home position
microscope.move((home_position_microscope))


#i_pipette=pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack=offset_pipette_rack)


sample_position_15welplate =  (300, 125, 132) 
move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)


In [ ]:
'''
calibrate reagetn slots
'''
pipette.home()


offset_x_reagents = -4
offset_y_reagents = 7
offset_z_reagents = -50 # smaller number means: go up

for i_pos in range(4):
    #i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, -8)
    position_reagent_i = np.array(plate_reagents.wells()[i_pos].center()._point)
    move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)

# move out of the well again
move_to_coord(pipette, position_reagent_i, offset=(0,0,20), minimum_z_height=minimum_z_height)

    

In [41]:
'''
calibrate illumination poisiton
'''
pipette_8.home()
#protocol.home()
#position_idle =  (100,300, 150)#
#move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)
#move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)
#microscope.home()
position_sample_light = (246, 99, 200) # height is necessary to reduce intensity
#Center of the center well
offset_x_a = 52490
offset_y_a = 48720
offset_z = microscope.position["z"]
move_to_coord(pipette_8, position_sample_light, offset=(0,0,20), minimum_z_height=minimum_z_height)
microscope.set_laser_led(0,1)
microscope.move((offset_x_a,offset_y_a,offset_z),absolute=True)
autofocus_dz = 1500
autofocus_Nz = 11
#offset_z = microscope.autofocus_coarse(dz=autofocus_dz, nz=autofocus_Nz)

microscope.capture_image_to_disk()
microscope.set_laser_led(0,0)    
microscope.set_laser_led(1000,0)
microscope.capture_image_to_disk()
microscope.set_laser_led(0,0)

if(0): #test laser 
    microscope.set_laser_led(0,1)
    autofocus_dz = 200
    autofocus_Nz = 7
    offset_z = autofocus_coarse(dz=autofocus_dz, nz=autofocus_Nz)
    
    # take BF image

    move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
    time.sleep(1) # debounce
    microscope.move((offset_x,offset_y,offset_z),absolute=True)
    microscope.capture_image_to_disk()

    # take Fluo image
    move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)
    time.sleep(1) # debounce
    microscope.set_laser_led(1,1)
    microscope.capture_image_to_disk()
    time.sleep(1)
    microscope.set_laser_led(0,0)

Please add a pipette first!
Moving to: (246, 99, 220)
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Setting Laser/LED to: 1000 0
Setting Laser/LED to: 0 0


In [25]:
# initialize strins


from datetime import datetime
timestamp = datetime.now().strftime("%Y_%m_%d-%I-%M-%S_%p")
i_experiment = 0


# start experiment

# Add PFA


In [52]:
if(0): # already done in the lab
    # move the wellplate to the zero position with the microscope
    microscope.move(home_position_microscope, absolute=True)

    # 1. Pickup new pipette
    i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)

    print(type(i_pipette))
    V_aspirate = 40

    # first go to sample and pickup liquid 
    move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.aspirate(100)

    # 3. Trash the liquid
    move_to_coord(pipette, position_trash, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents+10), minimum_z_height=minimum_z_height)

    # 3.1 get new pipette
    i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)

    # 4- Pickup BSa and bring it to sample         
    position_reagent_i = np.array(plate_reagents.wells()[i_pfa].center()._point)
    move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
    pipette.aspirate(V_aspirate)

    # 5. Go to sample and add BSA 
    move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.dispense()

    move_to_coord(pipette, sample_position_15welplate, offset=(0,0,15), minimum_z_height=minimum_z_height)


    # wait for 15 minutes 
    time.sleep(t_pfa*60)

In [ ]:
# 1. Pickup new pipette
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)

# move the wellplate to the zero position with the microscope
microscope.move(home_position_microscope, absolute=True)

print(type(i_pipette))
V_aspirate = 40

for _ in range(3):
    # first go to sample and pickup liquid 
    move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.aspirate()

    # 3. Trash the liquid
    move_to_coord(pipette, position_trash, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents+10), minimum_z_height=minimum_z_height)
    pipette.dispense()

    # 1. Pickup new pipette
    i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)

    # 4- Pickup Primary and bring it to sample         
    position_reagent_i = np.array(plate_reagents.wells()[i_pbs].center()._point)
    move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
    pipette.aspirate(V_aspirate)

    # 5. Go to sample and add PBS 
    move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.dispense()



# Add Triton and wash

In [56]:
pipette.home() 
# move the wellplate to the zero position with the microscope
microscope.move(home_position_microscope, absolute=True)

# 1. Pickup new pipette
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)

print(type(i_pipette))
V_aspirate = 40
        
# first go to sample and pickup liquid 
move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
pipette.aspirate(200)

# 3. Trash the liquid
move_to_coord(pipette, position_trash, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents+10), minimum_z_height=minimum_z_height)

# 3.1 get new pipette
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)

# 4- Pickup BSa and bring it to sample         
position_reagent_i = np.array(plate_reagents.wells()[i_triton].center()._point)
move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
pipette.aspirate(V_aspirate)

# 5. Go to sample and add BSA 
move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
pipette.dispense()
move_to_coord(pipette, sample_position_15welplate, offset=(0,0,15), minimum_z_height=minimum_z_height)

# wait for 15 minutes 
time.sleep(t_triton*60)

for _ in range(3):
    # first go to sample and pickup liquid 
    move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.aspirate(200)

    # 3. Trash the liquid
    move_to_coord(pipette, position_trash, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents+10), minimum_z_height=minimum_z_height)
    pipette.dispense()

    # 1. Pickup new pipette
    i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)

    # 4- Pickup Primary and bring it to sample         
    position_reagent_i = np.array(plate_reagents.wells()[i_pbs].center()._point)
    move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
    pipette.aspirate(V_aspirate)

    # 5. Go to sample and add PBS 
    move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.dispense()



My pipette number: 10and type: <class 'int'>
<class 'int'>
Moving to: (300, 125, 132)
Moving to: (361, 357, 60)
My pipette number: 11and type: <class 'int'>
Moving to: (34.1, 353.93, 10.949999999999996)
Moving to: (300, 125, 132)
Moving to: (300, 125, 132)
Moving to: (361, 357, 60)
My pipette number: 12and type: <class 'int'>
Moving to: (14.21, 315.37, 10.949999999999996)
Moving to: (300, 125, 132)
Moving to: (300, 125, 132)
Moving to: (361, 357, 60)
My pipette number: 13and type: <class 'int'>
Moving to: (14.21, 315.37, 10.949999999999996)
Moving to: (300, 125, 132)
Moving to: (300, 125, 132)
Moving to: (361, 357, 60)
My pipette number: 14and type: <class 'int'>
Moving to: (14.21, 315.37, 10.949999999999996)
Moving to: (300, 125, 132)


# Add Blocking Buffer

 we assume the sample is fixed and permeabilised and needs blocking fore primary/secondary antibody labelling

In [57]:

# move the wellplate to the zero position with the microscope
microscope.move(home_position_microscope, absolute=True)

# 1. Pickup new pipette
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)

print(type(i_pipette))
V_aspirate = 40
        
# first go to sample and pickup liquid 
move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
pipette.aspirate(200)

# 3. Trash the liquid
move_to_coord(pipette, position_trash, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents+10), minimum_z_height=minimum_z_height)

# 3.1 get new pipette
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)
                

# 4- Pickup BSa and bring it to sample         
position_reagent_i = np.array(plate_reagents.wells()[i_bsa].center()._point)
move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
pipette.aspirate(V_aspirate)

# 5. Go to sample and add BSA 
move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
pipette.dispense()


# go to idle positoin and rest
move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)

# wait for 15 minutes 
#time.sleep(15*60)

My pipette number: 15and type: <class 'int'>
<class 'int'>
Moving to: (300, 125, 132)
Moving to: (361, 357, 60)
My pipette number: 16and type: <class 'int'>
Moving to: (14.21, 296.09, 10.949999999999996)
Moving to: (300, 125, 132)
Moving to: (100, 100, 150)


# Do timelapse for 15 minutes 

In [76]:
# Microscope coordinates
from opentrons_helper_class import * 
is_autofocus = True

autofocus_dz = 1000
autofocus_Nz= 15
mytime = time.time()
t_duration = 20 # minutes
focus_pos_list = None # start with a fresh list
Nx=5
Ny=5
well_to_well_steps= 500
t_duration = 15 # how long does the experiment preform?
t_period = 1 # min - how often should the expimrent be carried out?
focus_pos_list = None
while (time.time()-mytime)<(t_bsa*60):
    
    # perform a testing well scan by moving robot to light position and do a whole plate scan
    move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
    focus_pos_list = wellscan(microscope, offset_x, offset_y, i_experiment,
            Nx, Ny, well_to_well_steps,
            autofocus_dz, autofocus_Nz,
                name_experiment="wellscan_afterbsa_"+timestamp,
                focus_pos_list=focus_pos_list, is_autofocus=is_autofocus,
                is_autofocus_fine = False, 
                is_autofocus_fast = False,
                I_laser=0, I_led=1)
    i_experiment += 1
    is_autofocus=False
    
    
    

#n_cells = imjoy.plugin("Countcells", image_stack)
#new_protocol = decide(n_cells)

Please add a pipette first!
Moving to: (246, 99, 200)
ATTENTION: DEPRECATED!!!
Start moving to the position
Start scan
Move microscope to (XY): 51990/46720
Setting Laser/LED to: 0 1
ERROR make sure the correct laser itnensity is assigned!


KeyboardInterrupt: 

# Add Primary

In [80]:
# move the wellplate to the zero position with the microscope
microscope.move(home_position_microscope, absolute=True)


print(type(i_pipette))
V_aspirate = 40
        
# first go to sample and pickup liquid 
move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
pipette.aspirate(200)

# 3. Trash the liquid
move_to_coord(pipette, position_trash, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents+10), minimum_z_height=minimum_z_height)
pipette.dispense()
                
# 1. Pickup new pipette
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)

# 4- Pickup Primary and bring it to sample         
position_reagent_i = np.array(plate_reagents.wells()[i_ab_primary].center()._point)
move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
pipette.aspirate(V_aspirate)

# 5. Go to sample and add BSA 
move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
pipette.dispense()


# go to idle positoin and rest
move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)


<class 'int'>
Moving to: (300, 125, 132)
Moving to: (361, 357, 60)
My pipette number: 22and type: <class 'int'>
Moving to: (14.21, 353.93, 10.949999999999996)
Moving to: (300, 125, 132)
Moving to: (100, 100, 150)


# Do timelapse for an hour

In [ ]:
# Microscope coordinates
mytime = time.time()
t_duration = 20 # minutes
focus_pos_list = None # start with a fresh list
Nx=5
Ny=5
well_to_well_steps= 500
t_duration = 30 # how long does the experiment preform?
t_period = 1 # min - how often should the expimrent be carried out?
move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)

while (time.time()-mytime)<(t_primary*60):
    
    # perform a testing well scan by moving robot to light position and do a whole plate scan
    move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
    focus_pos_list = wellscan(microscope, offset_x, offset_y, i_experiment,
            Nx, Ny, well_to_well_steps,
            autofocus_dz, autofocus_Nz,
                name_experiment="wellscan_afterprimary_"+timestamp,
                focus_pos_list=focus_pos_list, is_autofocus=is_autofocus,
                is_autofocus_fine = False, 
                is_autofocus_fast = False,
                I_laser=0, I_led=1)
    i_experiment += 1
    is_autofocus=False
    

Please add a pipette first!
Moving to: (246, 99, 200)
Please add a pipette first!
Moving to: (246, 99, 200)
ATTENTION: DEPRECATED!!!
Start moving to the position
Start scan
Move microscope to (XY): 51990/46720
Setting Laser/LED to: 0 1
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:475
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52490/46720
Setting Laser/LED to: 0 1
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52490/47220
Setting Laser/LED to: 0 1
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:476
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52990/47220
Setting Laser/LED to: 0 1
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:425
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY

offset_z:125
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53990/47220
offset_z:125
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 51990/47720
offset_z:143
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52490/47720
offset_z:345
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52990/47720
offset_z:45
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53490/47720
offset_z:45
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53990/47720
offset_z:139
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 51990/48220
offset_z:-1361
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52490/48220
offset_z:139
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52990/48220
offset_z:219
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope t

offset_z:143
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52490/47720
offset_z:345
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52990/47720
offset_z:45
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53490/47720
offset_z:45
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53990/47720
offset_z:139
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 51990/48220
offset_z:-1361
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52490/48220
offset_z:139
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52990/48220
offset_z:219
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53490/48220
offset_z:213
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53990/48220
offset_z:-87
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope t

offset_z:45
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53490/47720
offset_z:45
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53990/47720
offset_z:139
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 51990/48220
offset_z:-1361
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52490/48220
offset_z:139
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52990/48220
offset_z:219
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53490/48220
offset_z:213
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53990/48220
offset_z:-87
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 51990/48720
offset_z:-1587
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52490/48720
offset_z:-2992
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microsco

offset_z:139
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 51990/48220
offset_z:-1361
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52490/48220
offset_z:139
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52990/48220
offset_z:219
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53490/48220
offset_z:213
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53990/48220
offset_z:-87
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 51990/48720
offset_z:-1587
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52490/48720
offset_z:-2992
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52990/48720
offset_z:-2092
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53490/48720
offset_z:-1792
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move mi

...

offset_z:-1587
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52490/48720
offset_z:-2992
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52990/48720
offset_z:-2092
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53490/48720
offset_z:-1792
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53990/48720
offset_z:-1192
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Please add a pipette first!
Moving to: (246, 99, 200)
ATTENTION: DEPRECATED!!!
Start moving to the position
Start scan
Move microscope to (XY): 51990/46720
offset_z:475
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52490/46720
offset_z:395
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52990/46720
offset_z:95
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53490/46720
offset_z:95
Setting Laser/LED to: 0 1
Setting Laser/LED 

offset_z:-2092
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53490/48720
offset_z:-1792
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53990/48720
offset_z:-1192
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Please add a pipette first!
Moving to: (246, 99, 200)
ATTENTION: DEPRECATED!!!
Start moving to the position
Start scan
Move microscope to (XY): 51990/46720
offset_z:475
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52490/46720
offset_z:395
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52990/46720
offset_z:95
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53490/46720
offset_z:95
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53990/46720
offset_z:95
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 51990/47220
offset_z:176
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0

offset_z:-1192
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Please add a pipette first!
Moving to: (246, 99, 200)
ATTENTION: DEPRECATED!!!
Start moving to the position
Start scan
Move microscope to (XY): 51990/46720
offset_z:475
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52490/46720
offset_z:395
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52990/46720
offset_z:95
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53490/46720
offset_z:95
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53990/46720
offset_z:95
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 51990/47220
offset_z:176
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52490/47220
offset_z:476
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52990/47220
offset_z:425
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
M

Start scan
Move microscope to (XY): 51990/46720
offset_z:475
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52490/46720
offset_z:395
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52990/46720
offset_z:95
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53490/46720
offset_z:95
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53990/46720
offset_z:95
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 51990/47220
offset_z:176
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52490/47220
offset_z:476
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52990/47220
offset_z:425
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53490/47220
offset_z:125
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53990/47220
offset_z:125
Setting Laser/LED to: 0 

Setting Laser/LED to: 0 0
Move microscope to (XY): 52990/46720
offset_z:95
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53490/46720
offset_z:95
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53990/46720
offset_z:95
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 51990/47220
offset_z:176
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52490/47220
offset_z:476
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52990/47220
offset_z:425
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53490/47220
offset_z:125
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 53990/47220
offset_z:125
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 51990/47720
offset_z:143
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Move microscope to (XY): 52490/47720
offset_z:345
Setting L

# Wash sample 3x 

In [ ]:
# move the wellplate to the zero position with the microscope
microscope.move(home_position_microscope, absolute=True)


print(type(i_pipette))
V_aspirate = 40

for _ in range(3):
    # first go to sample and pickup liquid 
    move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.aspirate(200)

    # 3. Trash the liquid
    move_to_coord(pipette, position_trash, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents+10), minimum_z_height=minimum_z_height)
    pipette.dispense()

    # 1. Pickup new pipette
    i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)

    # 4- Pickup Primary and bring it to sample         
    position_reagent_i = np.array(plate_reagents.wells()[i_pbs].center()._point)
    move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
    pipette.aspirate(V_aspirate)

    # 5. Go to sample and add PBS 
    move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.dispense()


# go to idle positoin and rest
move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)

    # wait for 15 minutes 
    #time.sleep(15*60)

# Add Secondary 

In [ ]:
# move the wellplate to the zero position with the microscope
microscope.move(home_position_microscope, absolute=True)


print(type(i_pipette))
V_aspirate = 40
        
# first go to sample and pickup liquid 
move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
pipette.aspirate(200)

# 3. Trash the liquid
move_to_coord(pipette, position_trash, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents+10), minimum_z_height=minimum_z_height)
pipette.dispense()

# 1. Pickup new pipette
i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)
               

# 4- Pickup Primary and bring it to sample         
position_reagent_i = np.array(plate_reagents.wells()[i_ab_secondary].center()._point)
move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
pipette.aspirate(V_aspirate)

# 5. Go to sample and add Secondary 
move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
pipette.dispense()

# go to idle positoin and rest
move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)

# wait for 15 minutes 
#time.sleep(t_secondary*60)

# Do time lapse but with larger timegaps to not destroy the fluorophore

In [ ]:
# Microscope coordinates

mytime = time.time()
t_duration = 20 # minutes
focus_pos_list = None # start with a fresh list
Nx=8
Ny=3
well_to_well_steps= 500
t_duration = 60 # how long does the experiment preform?
t_period = 10 # min - how often should the expimrent be carried out?
move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
      
##
mytime2 = -1
while (time.time()-mytime)<(t_secondary*60):
    mytime = time.time()
    if (time.time()-mytime2)>t_period:
        mytime2=time.time()
    

        # perform a testing well scan by moving robot to light position and do a whole plate scan
        move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
        focus_pos_list = wellscan(microscope, offset_x, offset_y, i_experiment,
                Nx, Ny, well_to_well_steps,
                autofocus_dz, autofocus_Nz,
                    name_experiment="wellscan_aftersecondary_"+timestamp,
                    focus_pos_list=focus_pos_list, is_autofocus=is_autofocus,
                    is_autofocus_fine = False, 
                    is_autofocus_fast = False,
                    I_laser=0, I_led=1)
        move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)

        i_experiment += 1

# Wash 3x with PBS

In [ ]:
# move the wellplate to the zero position with the microscope
microscope.move(home_position_microscope, absolute=True)

# 1. Pickup new pipette
#i_pipette = pickup_fresh_pipette_tip(i_pipette, pipette, tiprack, offset_pipette_rack)
V_aspirate = 60

for _ in range(3):
    # first go to sample and pickup liquid 
    move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.aspirate()

    # 3. Trash the liquid
    move_to_coord(pipette, position_trash, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents+10), minimum_z_height=minimum_z_height)
    pipette.dispense()


    # 4- Pickup Primary and bring it to sample         
    position_reagent_i = np.array(plate_reagents.wells()[i_pbs].center()._point)
    move_to_coord(pipette, position_reagent_i, offset=(offset_x_reagents,offset_y_reagents,offset_z_reagents), minimum_z_height=minimum_z_height)
    pipette.aspirate(V_aspirate)

    # 5. Go to sample and add PBS 
    move_to_coord(pipette, sample_position_15welplate, offset=(0,0,0), minimum_z_height=minimum_z_height)
    pipette.dispense()


# go to idle positoin and rest
move_to_coord(pipette, position_idle, offset=(0,0,0), minimum_z_height=minimum_z_height)

    # wait for 15 minutes 
    #time.sleep(15*60)

# Do timelapse

In [34]:
# Microscope coordinates
i_experiment+=1
mytime = time.time()
t_duration = 20 # minutes
focus_pos_list = None # start with a fresh list
Nx=15
Ny=25
well_to_well_steps= 500
t_duration = 60 # how long does the experiment preform?
t_period = 1 # min - how often should the expimrent be carried out?
autofocus_dz, autofocus_Nz=500,15
##
focus_pos_list=None
is_autofocus=True
# perform a testing well scan by moving robot to light position and do a whole plate scan
move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
focus_pos_list = wellscan(microscope, offset_x, offset_y, i_experiment,
        Nx, Ny, well_to_well_steps,
        autofocus_dz, autofocus_Nz,
            name_experiment="HeLa_overnight_longtime_Microtubule_"+timestamp,
            focus_pos_list=focus_pos_list, is_autofocus=is_autofocus,
            is_autofocus_fine = False, 
            is_autofocus_fast = False,
            I_laser=0, I_led=1)

# perform a testing well scan by moving robot to light position and do a whole plate scan
move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
focus_pos_list = wellscan(microscope, offset_x, offset_y, i_experiment,
        Nx, Ny, well_to_well_steps,
        autofocus_dz, autofocus_Nz,
            name_experiment="HeLa_overnight_longtime_Microtubule_"+timestamp,
            focus_pos_list=focus_pos_list, is_autofocus=is_autofocus,
            is_autofocus_fine = False, 
            is_autofocus_fast = False,
            I_laser=100, I_led=0)

Please add a pipette first!
Moving to: (246, 99, 200)
ATTENTION: DEPRECATED!!!
Start moving to the position
Start scan
Move microscope to (XY): 51990/47720
Setting Laser/LED to: 0 1
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:-358
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 52490/47720
Setting Laser/LED to: 0 1
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:-298
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 52990/47720
Setting Laser/LED to: 0 1
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:-307
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 53490/47720
Setting Laser/LED to: 0 1
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:-329
Setting

ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:-367
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 54490/48720
Setting Laser/LED to: 0 1
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:-411
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 54990/48720
Setting Laser/LED to: 0 1
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:-432
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 55490/48720
Setting Laser/LED to: 0 1
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:-379
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 55990/48720
Setting Laser/LED to: 0 1
ERROR make sure the correct laser itnensity is a

Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 56490/49720
Setting Laser/LED to: 0 1
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:589
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 56990/49720
Setting Laser/LED to: 0 1
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:1128
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 57490/49720
Setting Laser/LED to: 0 1
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:1055
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 57990/49720
Setting Laser/LED to: 0 1
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:1063
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0

ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:-124
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 58990/50720
Setting Laser/LED to: 0 1
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:42
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 51990/51220
Setting Laser/LED to: 0 1
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:-392
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 52490/51220
Setting Laser/LED to: 0 1
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:-292
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 52990/51220
Setting Laser/LED to: 0 1
ERROR make sure the correct laser itnensity is ass

Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 53490/52220
Setting Laser/LED to: 0 1
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:1020
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 53990/52220
Setting Laser/LED to: 0 1
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:1022
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 54490/52220
Setting Laser/LED to: 0 1
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:1030
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 54990/52220
Setting Laser/LED to: 0 1
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:1004
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Setting Laser/LED to: 

...

Setting Laser/LED to: 0 0
offset_z:1722
Setting Laser/LED to: 100 0
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 58990/54220
Setting Laser/LED to: 100 0
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:2063
Setting Laser/LED to: 100 0
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 51990/54720
Setting Laser/LED to: 100 0
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:2166
Setting Laser/LED to: 100 0
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 52490/54720
Setting Laser/LED to: 100 0
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:2666
Setting Laser/LED to: 100 0
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 52990/54720
Setting Laser/LED to: 100 0
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset

Setting Laser/LED to: 0 0
offset_z:1293
Setting Laser/LED to: 100 0
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 53490/55720
Setting Laser/LED to: 100 0
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:1793
Setting Laser/LED to: 100 0
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 53990/55720
Setting Laser/LED to: 100 0
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:1864
Setting Laser/LED to: 100 0
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 54490/55720
Setting Laser/LED to: 100 0
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:2078
Setting Laser/LED to: 100 0
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 54990/55720
Setting Laser/LED to: 100 0
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset

Setting Laser/LED to: 0 0
offset_z:2414
Setting Laser/LED to: 100 0
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 55490/56720
Setting Laser/LED to: 100 0
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:2078
Setting Laser/LED to: 100 0
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 55990/56720
Setting Laser/LED to: 100 0
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:2372
Setting Laser/LED to: 100 0
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 56490/56720
Setting Laser/LED to: 100 0
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:2586
Setting Laser/LED to: 100 0
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 56990/56720
Setting Laser/LED to: 100 0
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset

Setting Laser/LED to: 0 0
offset_z:2749
Setting Laser/LED to: 100 0
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 57490/57720
Setting Laser/LED to: 100 0
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:2249
Setting Laser/LED to: 100 0
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 57990/57720
Setting Laser/LED to: 100 0
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:2606
Setting Laser/LED to: 100 0
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 58490/57720
Setting Laser/LED to: 100 0
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:3106
Setting Laser/LED to: 100 0
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 58990/57720
Setting Laser/LED to: 100 0
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset

Setting Laser/LED to: 0 0
offset_z:-55
Setting Laser/LED to: 100 0
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 51990/59220
Setting Laser/LED to: 100 0
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:230
Setting Laser/LED to: 100 0
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 52490/59220
Setting Laser/LED to: 100 0
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:444
Setting Laser/LED to: 100 0
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 52990/59220
Setting Laser/LED to: 100 0
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:586
Setting Laser/LED to: 100 0
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 53490/59220
Setting Laser/LED to: 100 0
ERROR make sure the correct laser itnensity is assigned!
Setting Laser/LED to: 0 0
offset_z:6

In [46]:
# Microscope coordinates
i_experiment+=1
mytime = time.time()
t_duration = 20 # minutes
focus_pos_list = None # start with a fresh list
Nx=5
Ny=5
well_to_well_steps= 500
t_duration = 60 # how long does the experiment preform?
t_period = 1 # min - how often should the expimrent be carried out?
autofocus_dz, autofocus_Nz=300,5
##
focus_pos_list=None
is_autofocus=False
# perform a testing well scan by moving robot to light position and do a whole plate scan
move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
focus_pos_list = wellscan(microscope, offset_x, offset_y, i_experiment,
        Nx, Ny, well_to_well_steps,
        autofocus_dz, autofocus_Nz,
            name_experiment="HeLa_overnight_longtime_Microtubule_final_"+timestamp,
            focus_pos_list=focus_pos_list, is_autofocus=is_autofocus,
            is_autofocus_fine = False, 
            is_autofocus_fast = False,
            I_laser=1000, I_led=0)


Please add a pipette first!
Moving to: (246, 99, 200)
ATTENTION: DEPRECATED!!!
Start moving to the position
Start scan
Move microscope to (XY): 51990/47720
list index out of range
offset_z:-204
Setting Laser/LED to: 1000 0
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 52490/47720
list index out of range
offset_z:-204
Setting Laser/LED to: 1000 0
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 52990/47720
list index out of range
offset_z:-204
Setting Laser/LED to: 1000 0
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 53490/47720
list index out of range
offset_z:-204
Setting Laser/LED to: 1000 0
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 53990/47720
list index out of range
offset_z:-204
Setting Laser/LED to: 1000 0
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 51990/48220
list index out of range
offset_z:-204
Setting Laser/LED to: 1000

In [47]:
# Microscope coordinates
i_experiment+=1
mytime = time.time()
t_duration = 20 # minutes
focus_pos_list = None # start with a fresh list
Nx=5
Ny=5
well_to_well_steps= 500
t_duration = 60 # how long does the experiment preform?
t_period = 1 # min - how often should the expimrent be carried out?
autofocus_dz, autofocus_Nz=300,5
##
focus_pos_list=None
is_autofocus=False
# perform a testing well scan by moving robot to light position and do a whole plate scan
move_to_coord(pipette_8, position_sample_light, offset=(0,0,0), minimum_z_height=minimum_z_height)
focus_pos_list = wellscan(microscope, offset_x, offset_y, i_experiment,
        Nx, Ny, well_to_well_steps,
        autofocus_dz, autofocus_Nz,
            name_experiment="HeLa_overnight_longtime_Microtubule_final_"+timestamp,
            focus_pos_list=focus_pos_list, is_autofocus=is_autofocus,
            is_autofocus_fine = False, 
            is_autofocus_fast = False,
            I_laser=0, I_led=1)


Please add a pipette first!
Moving to: (246, 99, 200)
ATTENTION: DEPRECATED!!!
Start moving to the position
Start scan
Move microscope to (XY): 51990/47720
list index out of range
offset_z:-204
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 52490/47720
list index out of range
offset_z:-204
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 52990/47720
list index out of range
offset_z:-204
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 53490/47720
list index out of range
offset_z:-204
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 53990/47720
list index out of range
offset_z:-204
Setting Laser/LED to: 0 1
Setting Laser/LED to: 0 0
Setting Laser/LED to: 0 0
Move microscope to (XY): 51990/48220
list index out of range
offset_z:-204
Setting Laser/LED to: 0 1
Setting Laser/L